In [6]:
#モジュールを自動リロード
%load_ext autoreload
%autoreload 2

from LLMSearch.RWKVEmbedding import RWKVEmbedding

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
import json
with open('settings/settings.json') as f:
    settings = json.load(f)


In [182]:
embedder=RWKVEmbedding(settings)

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading /media/kh/python/2023/RWKV-4b-Pile-3B-20230228-7963.pth ...
Strategy: (total 32+1=33 layers)
* cpu [float32, float32], store 33 layers
0-cpu-float32-float32 1-cpu-float32-float32 2-cpu-float32-float32 3-cpu-float32-float32 4-cpu-float32-float32 5-cpu-float32-float32 6-cpu-float32-float32 7-cpu-float32-float32 8-cpu-float32-float32 9-cpu-float32-float32 10-cpu-float32-float32 11-cpu-float32-float32 12-cpu-float32-float32 13-cpu-float32-float32 14-cpu-float32-float32 15-cpu-float32-float32 16-cpu-float32-float32 17-cpu-float32-float32 18-cpu-float32-float32 19-cpu-float32-float32 20-cpu-float32-float32 21-cpu-float32-float32 22-cpu-float32-float32 23-cpu-float32-float32 24-cpu-float32-float32 25-cpu-float32-float32 26-cpu-float32-float32 27-cpu-float32-float32 28-cpu-float32-float32 29-cpu-float32-float32 30-cpu-float32-float32 31-cpu-float32-float32 32-cpu-float32-float32 
emb.weight                        f32      cpu  50277  2560 


In [207]:
from LLMSearch.TextSplitter import auto_txt_split

glob_path=settings["data_path"] + "/texts/*.txt"
chunk_size_limit= settings["chunk_size_limit"]
auto_txt_split(glob_path, chunk_size_limit,init=True)

UserData/split/ginga_tets.txt_chunk_0.txt
UserData/split/ginga_tets.txt_chunk_1.txt
UserData/split/ginga_tets.txt_chunk_2.txt
UserData/split/ginga_tets.txt_chunk_3.txt
UserData/split/wagahai.txt_chunk_0.txt
UserData/split/wagahai.txt_chunk_1.txt


In [215]:
#chunk_path_listからそれぞれデータを読み込み､RWKVによる埋め込みを行う
import numpy as np
from LLMSearch.TextSearcher import TextSearcher
from tqdm import tqdm
import glob


chunk_path_list=glob.glob(settings["data_path"] + "/split/*.txt")
searcher=TextSearcher(embedder,settings["data_path"])

#searcher.embedder.mean_mode=False

for path in tqdm(chunk_path_list):
    searcher.calc_text_file(path)

100%|██████████| 6/6 [00:07<00:00,  1.18s/it]


In [209]:
searcher.save_model()
searcher.load_model()

In [223]:
txt="I am a cat "
txt="猫である"
#txt="吾輩は猫である "*10
#txt="吾輩は猫である。名前はまだ無い。"
#txt="あなたはわかっているでしょう"
k=3

searcher.search(txt,k)

[{'path': 'UserData/split/wagahai.txt_chunk_0.txt',
  'text': '吾輩は猫である。名前はまだ無い。',
  'sim': 0.81906796},
 {'path': 'UserData/split/ginga_tets.txt_chunk_2.txt',
  'text': 'ところが先生は早くもそれを見附けたのでした。',
  'sim': 0.79416037},
 {'path': 'UserData/split/ginga_tets.txt_chunk_3.txt',
  'text': '「ジョバンニさん。あなたはわかっているのでしょう。」',
  'sim': 0.776914}]

In [219]:
txt

'I am a catI am a catI am a cat'